In this notebook, we use the hypotheses formed when we ran our EDA to conduct some hypothesis tests. We are most interested in whether or not a post in a certain category have significantly higher views/subscribes or engagement (likes + comments/views) on average. 

From our EDA on the training data, we formed the following hypotheses:

1) Posts that use at least one hashtag in either the title or descriptio have significantly higher views/subscriber
2) Posts with an affiliate link have significantly lower views/subscriber.
3) Posts that declare sponsorship in the description (not the title) have significantly lower views/subscriber
4) Posts from users that are verified have significantly higher engagement
5) Posts from users that are verified have significantly lower views/subscriber
6) Posts that mention a popular brand have significantly higher views/subscriber
7) Posts that used keywords from the "korean" keyword group have higher views/subscriber...
8) ...as well as higher engagement
9) Posts that were posted during a "prime hour" have significantly higher views/subcriber
10) Posts that were posted during a "prime hour" have significantly higher engagement. 

We will run several t-tests to confirm if these hypotheses are true or not on the testing set. To account for the fact that we are running multiple hypothesis tests, we will lower our significance level from the standard alpha = 0.05 to alpha = 0.01. This means we will have a 10% chance of a type 1 error if we run 10 tests.